### rlhf dataflow

```
actor, critic, reward, ref = initialize_models()
```

- https://github.com/zhaochenyang20/Awesome-ML-SYS-Tutorial/blob/main/rlhf/verl/readme.md
    - actor 在 RLHF 会进行 auto-regressive decoding，而 critic, reward 和 reference 则只会 prefill，不会 decode。所以，我们将 actor 的推理特定称为 rollout，而其他模型的推理称为 inference。
- https://github.com/zhaochenyang20/Awesome-ML-SYS-Tutorial/blob/main/rlhf/OpenRLHF/readme.md#%E6%9B%B4%E6%96%B0%E6%B5%81%E7%A8%8B
    - 准备一个 batch 的 prompts；
    - 将这个 batch 的 prompts 输入给 **Actor**，解码得到 **responses**；
    - 将 prompt + responses 输入给 Critic/Reward/Reference，分别计算得得到所有 token 的 values、最后一个 token 的 reward 和所有 token 的 log probs，按照强化学习的术语，称这些数据为经验（experiences）了；
    - 根据 experiences 多轮计算 actor loss 和 critic loss 并更新 Actor 和 Critic 模型。
        - 对于第 4 步，我们当然可以一轮 experiences 就更新一次 actor 和 critic（on-policy），但是为了尽可能利用这个 batch 的 experiences，我们对 actor 和 critic 做多轮更新（off-policy）。我们将 experiences 中多轮更新开始前的 log probs 和 values 称为 old log probs 和 old values（reward 不会多轮计算）。在每一轮中，actor 和 critic 会生成 new log probs 和 new values，然后在 old 的基础上计算 actor loss 和 critic loss，然后更新参数。

- openrlhf

```python
# --------------------------------------------------------------
# 初始化RLHF中的四个模型
# --------------------------------------------------------------
actor, critic, reward, ref = initialize_models()

# --------------------------------------------------------------
# 训练
# --------------------------------------------------------------
# 对于每一个batch的数据
for i in steps: 
    # 先收集经验值
    exps = generate_experience(prompts, actor, critic, reward, ref)
    
    # 一个batch的经验值将被用于计算ppo_epochs次loss，更新ppo_epochs次模型
    # 这也意味着，当你计算一次新loss时，你用的是更新后的模型
    for j in ppo_epochs:
        actor_loss = cal_actor_loss(exps, actor)
        critic_loss = cal_critic_loss(exps, critic)
        
        actor.backward(actor_loss)
        actor.step()
        
        critc.backward(critic_loss)
        critic.step()
```

### controler

- 一个很自然的想法是用 single controller 来管理整个工作流程，然后每个子模块（Actor、Critic、Reward、Reference）再各自由一个 single controller 来管理。然而，这种 naive 的实现里，两层控制结构都是 single controller，因此系统内的通讯 overhead 非常大。一个可能不太直观的点是——最高层负责总调度的 single controller 承担的通讯压力，反而比每个子模块的 single controller 要小。

### hybrid engine

- 四个子模块分别需要什么 engine：
    - actor model 需要 **training engine** 和 **rollout engine**。前者是用现代 training engine，比如 Megatron 或者 FSDP，后者得用现代推理引擎，比如 SGLang 或者 vllm 作为 rollout engine。这里思考一个小问题，为什么不能拿着 training engine 得到的 logits 做 sampling 然后 decode，貌似也可以用去 rollout？简单来说，太慢了，用训练引擎做 decode 的效果自然不如专用的推理引擎。
    - critic model 需要 training engine 和 inference engine。前者还是是现代的训练引擎，但是后者，可以用现代的推理引擎的高效 prefill 来得到 value 么？其实不能，critic model 的 inference 会直接复用 training engine 的 forward 来得到 value，所以 critic 的 inference engine 和 training engine 其实是同一个。其中的原因在此旧事重提：
    - reference model 和 reward model 只需要 inference，因为二者不需要训练，但是如同我之前提到的一样，用现代推理引擎得到的 log probs 和 reward 的精度不如用现代训练引擎得到的精度，所以这里选择用 training engine 的 forward 来做 inference，得到 log probs 和 reward。

- 有了 single controller 和 multi-controller 的概念后，这里引入veRL 的第二个核心概念：**hybrid engine**。
    - 在 RLHF 流程中，actor model 的 **generation 和 rollout** 占据了绝大多数运行时间（在 veRL 是 58.9%）。并且，由于 **PPO 是 on-policy** 算法，经验（experiences）必须来自于被 train 的模型本身，
    - 因此，**rollout 和 training 是必须串行的**。如果这两者使用不同的资源组，比如 rollout 用 2 张卡，而 training 用 4 张卡，rollout 的时候 training 的资源闲置，training 的时候 rollout 的资源闲置，无论如何都会浪费大量的计算资源。由此，veRL 将 training 和 rollout engine 放置在同一个资源组中串行执行。training 时，将 rollout engine 的显存回收（offload 到 CPU 上 或者直接析构掉），rollout 时，再将 training engine 的显存释放掉。这种将 **actor model 的不同 engine** 放置在同一个资源组上的方案，就称为 hybrid engine。